### Chamando o SQL serveless para rodar a query

In [0]:
%run /Workspace/Users/c00010492@cassi.com.br/Demandas/Reembolso/script_personalizado/sqlStatementSqlWarehouse

In [0]:
def executeScript(sql):
    sql_api = SQLStatementAPI()
    return sql_api.execute_sql(sql)

### Tabela Prestador

In [0]:
sql = """

create or replace table mldata.default.medicamentos_imunobiologicos_injetaveis_tb_prestador as
               
SELECT DISTINCT

PRE.PRESTADOR AS COD_PRESTADOR,
PRE.HANDLE AS HANDLE_PRESTADOR,
PRE.CPFCNPJ AS CPF_CNPJ,
PRE.DATACREDENCIAMENTO AS DATA_CREDENCIAMENTO,
PRE.NOME AS PRESTADOR,
PRE.FANTASIA AS NOME_FANTASIA,
TPR.DESCRICAO AS TIPO_PRESTADOR,
ESP.DESCRICAO AS ESPECIALIDADE_PRESTADOR,
PREEND.LOGRADOURO,
PREEND.NUMERO,
PREEND.COMPLEMENTO,
PREEND.BAIRRO,
MUN.NOME AS MUNI_PREST,
E.SIGLA AS UF_PREST,
PREEND.CEP,
PREEND.DDDTELEFONE1||' - '||PREEND.TELEFONE1 AS TELEFONE1,
PREEND.DDDTELEFONE1||' - '||PREEND.TELEFONE1 AS TELEFONE2,
PREEND.DDDCELULAR||' - '||PREEND.CELULAR AS CELULAR

FROM silver.soc.SAM_PRESTADOR 				             PRE 
LEFT JOIN silver.soc.SAM_PRESTADOR_ENDERECO        PREEND ON PREEND.PRESTADOR = PRE.HANDLE
LEFT JOIN silver.soc.ESTADOS 					             E ON E.HANDLE = PRE.ESTADOPAGAMENTO
LEFT JOIN silver.soc.MUNICIPIOS 				           MUN ON MUN.HANDLE = PRE.MUNICIPIOPAGAMENTO
LEFT JOIN silver.soc.SAM_TIPOPRESTADOR 		         TPR ON PRE.TIPOPRESTADOR = TPR.HANDLE
JOIN silver.soc.SAM_PRESTADOR_ESPECIALIDADE        PESP ON PESP.PRESTADOR = PRE.HANDLE
JOIN silver.soc.SAM_ESPECIALIDADE                  ESP ON ESP.HANDLE = PESP.ESPECIALIDADE

WHERE
PRE.DATADESCREDENCIAMENTO IS NULL

ORDER BY
HANDLE_PRESTADOR
               
"""

executeScript(sql)

query será realizada no banco:SQLSCIENCE_001


,num_affected_rows,num_inserted_rows


### Tabela Participantes

In [0]:
sql = """

create or replace table mldata.default.medicamentos_imunobiologicos_injetaveis_tb_participante as
               
SELECT DISTINCT

--PARTICIPANTE E PLANO
GUI.GUIA,
BEN.NOME AS BENEFICIARIO,
BEN.MATRICULACASSI AS MATRICULA,
DATEDIFF(YEAR, MAT.DATANASCIMENTO, GETDATE()) AS IDADE,
LOWER(BEN.EMAIL) AS EMAIL,
DDDF.CODIGODDD ||' - '|| SEN.TELEFONE1 AS TELEFONE_FIXO,
DDDC.CODIGODDD ||' - '|| SEN.CELULAR AS TELEFONE_CELULAR,

CASE
	WHEN MAT.SEXO = 'F' THEN 'FEMININO'
	WHEN MAT.SEXO = 'M' THEN 'MASCULINO'
	ELSE 'OUTROS'
END AS SEXO,

SEN.LOGRADOURO,
SEN.NUMERO,
SEN.COMPLEMENTO,
SEN.BAIRRO,
MUN.NOME AS MUNICIPIO,
EST.SIGLA AS UF,
SEN.CEP, 
UPPER(SCGRUP.DESCRICAO) AS GRUPO_PLANO, 
CON.CONTRATO||' - '||UPPER(CON.CONTRATANTE) AS PLANO,

--PRESTADOR
PRE.HANDLE AS HANDLE_PRESTADOR,
PRE.PRESTADOR AS COD_PRESTADOR,
SRA.DESCRICAO AS TIPO_ATENDIMENTO,
SFA.DESCRICAO AS FINALIDADE_ATENDIMENTO,

--MEDICAMENTO/QUANTIDADE/CUSTO
GEV.DESCRICAOIMPORTADA AS MEDICAMENTO,
GEV.QTDAPRESENTADA AS QT_SOLICITADA,
GEV.QTDGLOSADA AS QT_GLOSADA,
GEV.QTDPAGTO AS QT_PAGA,
GEV.VALORAPRESENTADO AS TOTAL_PREVISTO,
(GEV.VALORAPRESENTADO - GEV.VALORGLOSADO) AS VALOR_PAGO,
AUT.DATAAUTORIZACAO AS DATA_AUTORIZACAO,
GEV.DATAATENDIMENTO AS DATA_ATENDIMENTO,
CLA.DESCRICAO,
UNI.DESCRICAO || '-' || UNI.DESCRICAODETALHADA AS UNIDADE_MED


FROM silver.soc.SAM_GUIA                                 GUI
JOIN silver.soc.SAM_GUIA_EVENTOS                         GEV ON GEV.GUIA = GUI.HANDLE
JOIN silver.soc.SAM_TGE                                  TGE ON TGE.HANDLE = GEV.EVENTO
JOIN silver.soc.SAM_CLASSEEVENTO 						             CLA ON CLA.HANDLE = TGE.CLASSEEVENTO
LEFT JOIN silver.soc.SAM_AUTORIZ                         AUT ON AUT.HANDLE = GUI.AUTORIZACAO
LEFT JOIN silver.soc.SAM_AUTORIZ_EVENTOSOLICIT           AES ON AES.AUTORIZACAO = AUT.HANDLE
LEFT JOIN silver.soc.SAM_AUTORIZ_EVENTOGERADO            EG  ON EG.EVENTOSOLICITADO = AES.HANDLE
JOIN silver.soc.SAM_TGE  							                  TGEG ON TGEG.HANDLE = EG.EVENTOGERADO
LEFT JOIN silver.soc.SAM_PRESTADOR                       PRE ON PRE.HANDLE = AUT.LOCALEXECUCAO
LEFT JOIN silver.soc.SAM_TIPOPRESTADOR 				         TPRES ON TPRES.HANDLE = PRE.TIPOPRESTADOR
LEFT JOIN silver.soc.SAM_REGIMEATENDIMENTO               SRA ON SRA.HANDLE = AUT.REGIMEATENDIMENTO
LEFT JOIN silver.soc.SAM_FINALIDADEATENDIMENTO 			     SFA ON SFA.HANDLE = AUT.FINALIDADEATENDIMENTO
JOIN silver.soc.SAM_BENEFICIARIO                         BEN ON BEN.HANDLE = AUT.BENEFICIARIO
JOIN silver.soc.SAM_MATRICULA 							             MAT ON MAT.HANDLE = BEN.MATRICULA
JOIN silver.soc.SAM_ENDERECO                             SEN ON SEN.HANDLE = BEN.ENDERECORESIDENCIAL
LEFT JOIN silver.soc.SAM_DDD 						                DDDF ON DDDF.HANDLE = SEN.DDDTELEFONE1
LEFT JOIN silver.soc.SAM_DDD 						                DDDC ON DDDC.HANDLE = SEN.DDDCELULAR
JOIN silver.soc.MUNICIPIOS 								               MUN ON MUN.HANDLE = SEN.MUNICIPIO
JOIN silver.soc.ESTADOS                   	   		   	   EST ON EST.HANDLE = SEN.ESTADO
JOIN silver.soc.SAM_CONTRATO                             CON ON CON.HANDLE = BEN.CONTRATO
JOIN silver.soc.SAM_GRUPOCONTRATO 											SCGRUP ON SCGRUP.HANDLE = CON.GRUPOCONTRATO
JOIN silver.soc.TIS_UNIDADEMEDIDA						            UNI ON UNI.HANDLE = GEV.UNIDADEMEDIDA


WHERE GEV.DATAATENDIMENTO >= '2021-01-01' --AND  GEV.DATAATENDIMENTO <= '2022-12-31'
AND CLA.DESCRICAO = '087 - IMUNOBIOLÓGICO INJETÁVEL'

ORDER BY 24
               
"""

executeScript(sql)

query será realizada no banco:SQLSCIENCE_001


,num_affected_rows,num_inserted_rows
